In [144]:
%matplotlib inline
import tensorflow as tf
import numpy as np

In [149]:
W = tf.Variable([0, 0, 0], dtype=tf.float32)
b = tf.Variable([0], dtype=tf.float32)

x = tf.placeholder(tf.float32, shape=(None, 3))
y = tf.placeholder(tf.float32, shape=(None, 3))

learning_rate = 0.01
model = W * x + b
loss = (1 / 2) * tf.reduce_sum(tf.square(model - y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [156]:
X = np.array([
    [1, 2, 3],
    [2, 3, 4],
    [4, 5, 6],
])
Y = X * np.array([1.5, 0.5, 2.5])
steps = []
total_steps = 333

init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    for _ in range(total_steps):
        t, l, m = session.run([train, loss, model], feed_dict={x: X, y: Y})
        steps.append((l, m))

p = None
for l, m in steps:
    print('Loss: ' + str(l))
    p = m
print('---')
print(Y)
print('---')
print(p)

Loss: 219.0
Loss: 35.0475
Loss: 8.88722
Loss: 3.93677
Loss: 2.34923
Loss: 1.57848
Loss: 1.14034
Loss: 0.87951
Loss: 0.721146
Loss: 0.623216
Loss: 0.561145
Loss: 0.520414
Loss: 0.492422
Loss: 0.472067
Loss: 0.456319
Loss: 0.443382
Loss: 0.432185
Loss: 0.422093
Loss: 0.412727
Loss: 0.403862
Loss: 0.39536
Loss: 0.387141
Loss: 0.379153
Loss: 0.371367
Loss: 0.363762
Loss: 0.356326
Loss: 0.34905
Loss: 0.341926
Loss: 0.334951
Loss: 0.328119
Loss: 0.321429
Loss: 0.314874
Loss: 0.308454
Loss: 0.302165
Loss: 0.296004
Loss: 0.28997
Loss: 0.284057
Loss: 0.278266
Loss: 0.272593
Loss: 0.267035
Loss: 0.261591
Loss: 0.256257
Loss: 0.251033
Loss: 0.245915
Loss: 0.240901
Loss: 0.23599
Loss: 0.231178
Loss: 0.226465
Loss: 0.221848
Loss: 0.217325
Loss: 0.212894
Loss: 0.208554
Loss: 0.204302
Loss: 0.200136
Loss: 0.196056
Loss: 0.192059
Loss: 0.188143
Loss: 0.184307
Loss: 0.18055
Loss: 0.176868
Loss: 0.173263
Loss: 0.16973
Loss: 0.16627
Loss: 0.16288
Loss: 0.159559
Loss: 0.156306
Loss: 0.153119
Loss: 0.14999